## Importando bibliotecas necessárias ou instalando

ChartPole-v1 é um minijogo desenvolvido para o aprendizado de redes neurais e inteligência artificial. Basicamente, o objetivo do minijogo é estabilizar uma espécie de tronco na vertical apoiado em um bloco, atráves de 2 comandos: esquerda e direita.
Para mais informações o leitor pode acessar a [documentação](https://gymnasium.farama.org/environments/classic_control/cart_pole/) no gymnasium.

No caso imediatamente abaixo, não é necessário instalar no Kaggle, pois ele já vem com as bibliotecas necessárias por padrão. Mas eu montei esses comandos a fim de instalar no Jupyter Notebook do Anaconda 3. No ambiente do Kaggle eu treino a rede neural, pelo fato da GPU do servidor ser mais potente, enquanto executo o jogo no Jupyter Notebook, já que assim é possível renderizar o ambiente.

In [ ]:
!pip install gymnasium
!pip install "gymnasium[classic-control]"
!pip install stable-baselines3[extra]
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

A célula abaixo eu inseri para contornar um erro ao importar o PyTorch para o Jupyter Notebook no meu computador.

In [ ]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

Aqui são importadas as bibliotecas necessárias.

A rigor, só algumas dessas são necessárias, meu objetivo inicial no projeto era construir o modelo do 0, como no caso foi feito, mas eu não cheguei a implementar esse modelo no agente, mas deixei o esboço para projetos futuros.

In [ ]:
import gymnasium as gym
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
# from torch import nn, load, save
# from torch.optim import Adam

Aqui é carregado o ambiente com o minijogo usado.

Uma observação muito importante: como o notebook é treinado antes de tudo em um servidor (Colab ou Kaggle), então não foi necessário a sua renderização para visualização (até porque esse processo, que é possível, não é tão simples e tem pouca praticidade, devido às quedas no FPS). Sendo assim, o argumento *render_mode="human"* não foi inserido ao carregar o ambiente do minijogo.



In [ ]:
env = gym.make("CartPole-v1")
states = env.observation_space.shape[0] # numero de parâmetros na variável de observação do jogo
actions = env.action_space.n # número de ações possíveis = 2

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Abaixo eu apenas faço uma demonstração de como o jogo funciona, apenas para visualização.

O código abaixo não deve ser executado em um servidor (Colab/Kaggle). (a rigor, pode, mas não mudará em nada, você não poderá visualizar o minijogo, para isso, é necessário a instalação do Jupyter em ambiente local)

In [ ]:
episodes = 10
for i in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0

    while not done:
        env.render()
        action = np.random.choice([0, 1])
        n_state, reward, done, truncated, info = env.step(action)
        score += reward
    print(f"Episódio {i} concluído. Recompensa: {score}")

Episódio 1 concluído. Recompensa: 13.0
Episódio 2 concluído. Recompensa: 16.0
Episódio 3 concluído. Recompensa: 10.0
Episódio 4 concluído. Recompensa: 39.0
Episódio 5 concluído. Recompensa: 13.0
Episódio 6 concluído. Recompensa: 19.0
Episódio 7 concluído. Recompensa: 24.0
Episódio 8 concluído. Recompensa: 15.0
Episódio 9 concluído. Recompensa: 13.0
Episódio 10 concluído. Recompensa: 12.0


/usr/local/lib/python3.10/dist-packages/gymnasium/envs/classic_control/cartpole.py:215: UserWarning: WARN: You are calling render method without specifying any render mode. You can specify the render_mode at initialization, e.g. gym.make("CartPole-v1", render_mode="rgb_array")
  gym.logger.warn(


Bom, aqui ficaria o modelo de rede neural que eu gostaria de implementar nesse processo. Como disse, o esqueleto ficará dentro desse notebook para projetos futuros.

In [ ]:
# class modelo(nn.Module):
#     def __init__(self, state_size, action_size):
#         super().__init__()
#         self.flatten = nn.Flatten()
#         self.model = nn.Sequential(
#             nn.Linear(state_size, 24),
#             nn.ReLU(),
#             nn.Linear(24, 24),
#             nn.ReLU(),
#             nn.Linear(24, action_size)
#         )
#     def forward(self, x):
#         x = self.flatten(x)
#         return self.model(x)
# model = modelo(states, actions)

No código abaixo estará o algoritmo de Reinforcement Learning usado.

O algoritmo que eu usei foi o PPO, sigla para *Proximal Policy Optimization*. Ele é um algoritmo on-policy (somente usa os dados atuais, não passados, para tomada de decisões) cuja ideia é evitar mudanças bruscas na tomada de decisões do agente, ou seja, o agente irá tomar decisões próximas das que ele tomou anteriormente, evitando mudança abruptas. É um algortimo que pode usar ações discretas ou contínuas.

Eu usei a biblioteca StableBaselines3 para usar esse algoritmo. Outro que pode ser testado é o DQN (que no caso, é off-policy, ou seja, usa dados do passado). Entretanto, o PPO apresentou resultados satisfatórios para mim.

Segue abaixo alguns links úteis:

[Documentação do PPO no StableBaselines3](https://stable-baselines3.readthedocs.io/en/master/modules/ppo.html)

[Lista de algoritmos disponíveis](https://stable-baselines3.readthedocs.io/en/master/guide/algos.html)



In [ ]:
from stable_baselines3 import PPO
model = PPO('MlpPolicy', env, verbose=1)
model.learn(total_timesteps=50000, progress_bar=True)

Output()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 21.9     |
|    ep_rew_mean     | 21.9     |
| time/              |          |
|    fps             | 712      |
|    iterations      | 1        |
|    time_elapsed    | 2        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 25.5        |
|    ep_rew_mean          | 25.5        |
| time/                   |             |
|    fps                  | 571         |
|    iterations           | 2           |
|    time_elapsed         | 7           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.008688335 |
|    clip_fraction        | 0.0906      |
|    clip_range           | 0.2         |
|    entropy_loss  

Abaixo iremos salvar o modelo que foi treinado no servidor.

In [ ]:
model.save("PPO.pt")

E no código abaixo é a seção que deve ser executada em ambiente local, com o modelo já salvo que deverá ser carregado para visualização

In [ ]:
model = PPO.load("PPO.pt")
episodes = 100
for i in range(1, episodes+1):
    done = False
    score = 0
    obs = env.reset()[0]

    while not done:
        env.render()
        action, _ = model.predict(obs)
        obs, reward, done, truncated, info = env.step(action)
        score += reward
    print(f"Episódio {i} concluído. Recompensa: {score}")

Abaixo está um exemplo de como ficou esse modelo sendo executado em ambiente local

In [11]:
from IPython.display import HTML
from base64 import b64encode

def play(filename):
    html = ''
    video = open(filename,'rb').read()
    src = 'data:video/mp4;base64,' + b64encode(video).decode()
    html += '<video width=1000 controls autoplay loop><source src="%s" type="video/mp4"></video>' % src 
    return HTML(html)

play('/kaggle/input/testando/pygame window 2025-01-06 01-29-15.mp4')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
